In [1]:
import os

folder = "data/cleaned_txt"
files = sorted([f for f in os.listdir(folder) if f.endswith(".txt")])


In [2]:
import re
from datetime import datetime

def ekstrak_metadata(teks):
    no_perkara = re.search(r"(Nomor|No\.)\s*Perkara[:\s]*([A-Za-z0-9\/\.\-]+)", teks)
    tanggal = re.search(r"\b(\d{1,2}[\-/]\d{1,2}[\-/]\d{4})\b", teks)
    pasal = re.findall(r"Pasal\s+\d+[A-Za-z]*", teks)
    pihak = re.findall(r"(Terdakwa|Penggugat|Tergugat)\s*[:\-–]\s*([A-Z][^\n\r]+)", teks)

    return {
        "no_perkara": no_perkara.group(2) if no_perkara else "",
        "tanggal": tanggal.group(1) if tanggal else "",
        "pasal": "; ".join(set(pasal)),
        "pihak": "; ".join([f"{p[0]}: {p[1].strip()}" for p in pihak]),
    }


In [3]:
def ekstrak_fakta(teks):
    match = re.search(r"(menimbang\s+bahwa.*?)((menimbang\s+bahwa|menurut\s+hukum|amar)|$)", teks, re.IGNORECASE | re.DOTALL)
    return match.group(1).strip() if match else ""

def ekstrak_argumen(teks):
    match = re.search(r"(menurut\s+hukum.*?)(amar\s+putusan|demikian|putusan)", teks, re.IGNORECASE | re.DOTALL)
    return match.group(1).strip() if match else ""


In [4]:
def fitur_teks(teks):
    return {
        "length": len(teks.split()),  # jumlah kata
        "has_pasal": "pasal" in teks.lower(),
    }


In [5]:
import pandas as pd

data = []
for i, file in enumerate(files):
    with open(os.path.join(folder, file), "r", encoding="utf-8") as f:
        teks = f.read()

    meta = ekstrak_metadata(teks)
    fakta = ekstrak_fakta(teks)
    argumen = ekstrak_argumen(teks)
    fitur = fitur_teks(teks)

    data.append({
        "case_id": i + 1,
        "filename": file,
        "no_perkara": meta["no_perkara"],
        "tanggal": meta["tanggal"],
        "pasal": meta["pasal"],
        "pihak": meta["pihak"],
        "ringkasan_fakta": fakta[:500],  # potong biar ringkas
        "argumen_hukum": argumen[:500],
        "text_full": teks[:1000],
        "length": fitur["length"],
        "has_pasal": fitur["has_pasal"],
    })

# Simpan ke CSV dan JSON
os.makedirs("data/processed", exist_ok=True)
pd.DataFrame(data).to_csv("data/processed/cases.csv", index=False)
pd.DataFrame(data).to_json("data/processed/cases.json", orient="records", indent=2)


In [6]:
import os
import re
import pandas as pd

folder_txt = "data/cleaned_txt"
files = sorted([f for f in os.listdir(folder_txt) if f.endswith(".txt")])

def ekstrak_metadata(teks):
    # Nomor perkara
    no_perkara = re.search(r"(nomor|no\.)\s*perkara[:\s]*([A-Za-z0-9\/\.\-]+)", teks, re.IGNORECASE)
    
    # Tanggal
    tanggal = re.search(r"\b(\d{1,2}[-/]\d{1,2}[-/]\d{4})\b", teks)
    
    # Pasal
    pasal = re.findall(r"pasal\s+\d+[A-Za-z]*\s*(?:ayat\s*\([^)]+\))?", teks, re.IGNORECASE)
    
    # Nama pihak (penggugat/tergugat/terdakwa)
    pihak = re.findall(r"(terdakwa|penggugat|tergugat)[:\-–]?\s*([A-Z][^\n\r]+)", teks, re.IGNORECASE)
    
    return {
        "no_perkara": no_perkara.group(2) if no_perkara else "",
        "tanggal": tanggal.group(1) if tanggal else "",
        "pasal": "; ".join(set(pasal)),
        "pihak": "; ".join([f"{p[0].capitalize()}: {p[1].strip()}" for p in pihak])
    }

def ekstrak_ringkasan_fakta(teks):
    match = re.search(r"(menimbang\s+bahwa.*?)(menimbang|mengingat|amar)", teks, re.IGNORECASE | re.DOTALL)
    return match.group(1).strip() if match else teks[:700]

# Gabungkan semuanya
case_data = []

for idx, file in enumerate(files):
    with open(os.path.join(folder_txt, file), "r", encoding="utf-8") as f:
        teks = f.read()
    
    metadata = ekstrak_metadata(teks)
    ringkasan = ekstrak_ringkasan_fakta(teks)
    
    case_data.append({
        "case_id": idx + 1,
        "no_perkara": metadata["no_perkara"],
        "tanggal": metadata["tanggal"],
        "ringkasan_fakta": ringkasan,
        "pasal": metadata["pasal"],
        "pihak": metadata["pihak"],
        "text_full": teks[:3000]  # atau simpan seluruh teks
    })

# Simpan ke CSV
os.makedirs("data/processed", exist_ok=True)
df = pd.DataFrame(case_data)
df.to_csv("data/processed/cases.csv", index=False, encoding="utf-8-sig")


In [7]:
for f in os.listdir("data/cleaned_txt"):
    with open(os.path.join("data/cleaned_txt", f), encoding="utf-8") as file:
        isi = file.read()
        print(f"\n==== {f} ====\n", isi[:1000])  # tampilkan awal isi
        break  # hanya 1 file untuk lihat



==== case (1).txt ====
 direktori putusan mahkamah agung republik indonesia putusanmahkamahagunggoid mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia disclaimer kepaniteraan mahkamah agung republik indonesia berusaha mencantumkan informasi akurat bentuk komitmen mahkamah agung pelayanan publik transparansi akuntabilitas pelaksanaan fungsi peradilan halhal permasalahan teknis terkait akurasi keterkinian informasi sajikan perbaiki kewaktu menemukan inakurasi informasi termuat situs informasi tersedia harap hubungi kepaniteraan mahkamah agung email kepaniteraanmahkamahagunggoid telp ext halaman putusan nomor kpidsus nomor kpidsus keadilan berdasarkan ketuhanan maha esa memeriksa perkara tindak pidana khusus tingkat kasasi dimohonkan penuntut kejaksaan negeri jakarta selatan terdakwa memutus perkara terdakwa nama fadlan djakfar lahir aceh umurtanggal lahir april jenis k

In [1]:
import pdfplumber

def ambil_header_pdf(path_pdf):
    with pdfplumber.open(path_pdf) as pdf:
        first_page = pdf.pages[0]
        teks = first_page.extract_text()
        return teks


In [3]:
import os

In [4]:
folder_mentah = "data/extracted_texts"
for f in os.listdir(folder_mentah):
    with open(os.path.join(folder_mentah, f), encoding="utf-8") as file:
        print(f, file.read()[:1000])  # lihat apakah metadata masih ada


case (1).txt Direktori Putusan Mahkamah Agung Republik Indonesia
putusan.mahkamahagung.go.id

Mahkamah Agung Republik Indonesia
Mahkamah Agung Republik Indonesia
Mahkamah Agung Republik Indonesia
Mahkamah Agung Republik Indonesia
Mahkamah Agung Republik Indonesia

Disclaimer
Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas
pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu.
Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :
Email : kepaniteraan@mahkamahagung.go.id    Telp : 021-384 3348 (ext.318)

Halaman 1

Hal

In [9]:
import os

folder_raw = "data/extracted_texts"

for f in os.listdir(folder_raw):
    if f.endswith(".txt"):
        with open(os.path.join(folder_raw, f), encoding="utf-8") as file:
            teks = file.read()
            no_perkara = ekstrak_nomor_perkara(teks)
            print(f"{f}: {no_perkara}")


case (1).txt: 39 K/Pid.Sus/2022
case (10).txt: 164/Pid.Sus/2023
case (11).txt: 267/Pid.Sus/2020
case (12).txt: 15/2014 tanggal 10 Juli 2014 (BB A.44); 22) 1 (satu) set fotokopi pengesahan akta pendirian dari Kemenkumham Nomor AHU-20703.40.10.2014 (BB A.45); 23) 1 (satu) set fotokopi Akta Salinan Keputusan Rapat Nomor 15/2016
case (13).txt: 1147K/Pid.Sus/2022
case (14).txt: 460/Pid.Sus/2023
case (15).txt: 1503 K/Pid.Sus/2025
case (16).txt: 1506 K/Pid.Sus/2023
case (17).txt: 2150K/Pid.Sus/2025
case (18).txt: 2694 K/Pid.Sus/2022
case (19).txt: 3004 K/Pid.Sus/2023
case (2).txt: 82/Pid.Sus/2021
case (20).txt: 3235 K/Pid.Sus/2023
case (21).txt: 3694 K/Pid.Sus/2023
case (22).txt: 3845 K/Pid.Sus/2022
case (23).txt: 4217 K/Pid.Sus/2020
case (24).txt: 5513 K/Pid.Sus/2024
case (25).txt: 5673 K/Pid.Sus/2023
case (26).txt: 5848 K/Pid.Sus/2023
case (27).txt: 6003 K/Pid.Sus/2022
case (28).txt: 6958 K/Pid.Sus/2024
case (29).txt: 6981 K/Pid.Sus/2024
case (3).txt: 337/Pid.Sus/2020
case (30).txt: 7728 K/

In [7]:
import re

def ekstrak_nomor_perkara(teks):
    """
    Mengekstrak nomor perkara dari teks mentah putusan.
    Mengembalikan string atau "" jika tidak ditemukan.
    """
    pola = re.compile(
        r"(putusan\s+)?(nomor|no\.?)\s*[:\-]?\s*([0-9]{1,4}\s*[A-Z]?\/[^/\n]+\/(?:TPK\/)?\d{4}(?:\/PN\.[A-Z\.]+)?)",
        re.IGNORECASE
    )
    match = pola.search(teks)
    return match.group(3).strip() if match else ""


In [8]:
# Contoh teks mentah
teks = """
MAHKAMAH AGUNG REPUBLIK INDONESIA
Putusan Nomor 39 K/Pid.Sus/2022

Mengadili...
"""

print(ekstrak_nomor_perkara(teks))
# Output: 39 K/Pid.Sus/2022


39 K/Pid.Sus/2022


In [ ]:
[ {
    "query": "Terdakwa tidak menyampaikan SPT Tahunan selama 3 tahun berturut-turut dan menyebabkan kerugian pada pendapatan negara.",
    "ground_truth": ["case(1).txt"]
  },
  {
    "query": "Wajib Pajak didakwa memalsukan faktur pajak dan melanggar Pasal 39 ayat (1) huruf c UU KUP.",
    "ground_truth": ["case(4).txt"]
  },
  {
    "query": "Terdakwa menyampaikan SPT Masa PPN yang tidak benar sehingga merugikan negara.",
    "ground_truth": ["case(7).txt"]
  },
  {
    "query": "Tersangka menyuruh orang lain membuat faktur pajak fiktif untuk menghindari pembayaran PPN.",
    "ground_truth": ["case(12).txt"]
  },
  {
    "query": "Wajib pajak diduga dengan sengaja tidak menyetorkan pajak yang telah dipotong dari pihak lain.",
    "ground_truth": ["case(15).txt"]
  }
]

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json
import numpy as np

# 1. Initialize BERT model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 2. Define retrieve function
def retrieve(query, vectors, filenames, k=5):
    query_vec = model.encode(query)
    similarities = cosine_similarity([query_vec], vectors)[0]
    top_k_idx = similarities.argsort()[-k:][::-1]
    return [filenames[i] for i in top_k_idx]

# 3. Modified evaluation function to accept vectors and filenames
def evaluate_retrieval_bert(queries_json_path, vectors, filenames, k=5):
    with open(queries_json_path, "r", encoding="utf-8") as f:
        queries = json.load(f)

    total = len(queries)
    match = 0

    for q in queries:
        pred = retrieve(q["query"], vectors, filenames, k=k)
        if any(gt in pred for gt in q["ground_truth"]):
            match += 1

        print(f"\n🔍 Query: {q['query']}")
        print(f"🎯 Ground Truth: {q['ground_truth']}")
        print(f"🔁 Prediksi Top-{k}: {pred}")

    accuracy = match / total
    print(f"\n✅ Accuracy@{k}: {match}/{total} = {accuracy:.2%}")

documents = [
    # case (1).txt
    "Terdakwa Fadlan Djakfar didakwa sengaja membantu tindak pidana perpajakan berlanjut, yaitu menerbitkan faktur pajak berbarcode identitas Agung Kota Mandiri yang terbukti tidak berdasarkan transaksi sebenarnya, menyebabkan kerugian negara sebesar 96 juta rupiah lebih. Putusan kasasi memperbaiki denda.",
    
    # case (2).txt
    "Terdakwa Ida Laila binti Musripin, ketua kelompok usaha Rendingan, didakwa sengaja tidak menyetorkan Pajak Pertambahan Nilai (PPN) yang dipungut dari transaksi penjualan biji kopi. Kerugian pendapatan negara dihitung 10 miliar rupiah lebih. Terdakwa menjual aset tanah untuk menutupi denda.",
    
    # case (3).txt
    "Terdakwa Ir. Roni Wijaya, Direktur Keuangan PT Dutasari Citralaras, didakwa tindak pidana perpajakan dan tindak pidana pencucian uang. Terdakwa menginstruksikan pembelian dan penggunaan faktur pajak fiktif serta menyamarkan hasil tindak pidana untuk membeli aset properti. Putusan peninjauan kembali membatalkan putusan sebelumnya dan memerintahkan pengembalian kelebihan bayar denda.",
    
    # case (4).txt
    "Terdakwa Dian Ekawati, Komisaris PT Cahayasurya Timur, didakwa sengaja tidak menyetorkan SPT Tahunan dan memalsukan faktur pajak. Terdakwa terbukti melakukan pengkreditan faktur pajak fiktif tanpa transaksi sebenarnya, menyebabkan kerugian negara. Putusan kasasi menguatkan putusan pengadilan negeri dengan pidana penjara dan denda.",
    
    # case (5).txt
    "Terpidana Albert Lie, Direktur Karyaputra Lokatirta, didakwa pidana perpajakan karena menggunakan faktur pajak fiktif berdasarkan transaksi tidak sebenarnya (TBTS) yang dibeli dengan tujuan memperkecil pembayaran pajak. Putusan peninjauan kembali menolak permohonan terpidana dan menguatkan putusan sebelumnya.",
    
    # case (6).txt
    "Terdakwa Suminto bin Priyo Utomo didakwa menyediakan dan menjual barang kena cukai (rokok) jenis Sigaret Putih Mesin (SPM) merek Luffman American Blend tanpa dilekati pita cukai. Perbuatan ini menimbulkan kerugian negara. Putusan kasasi menolak permohonan penuntut dan menguatkan pidana yang dijatuhkan.",
    
    # case (7).txt
    "Terdakwa Mohamad Aly Shobat, Direktur Cahaya Firdaus, didakwa sengaja tidak menyetorkan Pajak Pertambahan Nilai (PPN) yang telah dipungut dari perusahaan seperti Pertamina. Putusan peninjauan kembali mengabulkan permohonan terpidana, membatalkan putusan sebelumnya, dan memerintahkan pengembalian kelebihan bayar denda karena kerugian negara telah dipulihkan.",
    
    # case (8).txt
    "Terdakwa Tri Anis Noorbaiti, mantan General Manager Finance Accounting Shields Indonesia, didakwa tidak melaporkan SPT lengkap dan tidak menyetorkan pajak yang dipotong/dipungut (PPN dan PPh Pasal 23 rekanan). Putusan peninjauan kembali menguatkan bahwa terpidana terbukti bersalah.",
    
    # case (9).txt
    "Terdakwa Ihmar, Direktur Andoyo Tofan Nugraha Abadi, didakwa sengaja menerbitkan faktur pajak berdasarkan transaksi tidak sebenarnya (TBTS) dan mengkreditkan SPT PPN fiktif, yang menyebabkan kerugian negara. Putusan kasasi menolak permohonan terpidana dan menguatkan putusan sebelumnya.",
]

filenames = [
    "case (1).txt",
    "case (2).txt",
    "case (3).txt",
    "case (4).txt",
    "case (5).txt",
    "case (6).txt",
    "case (7).txt",
    "case (8).txt",
    "case (9).txt"
]

# 5. Precompute document embeddings
vectors = model.encode(documents)

# 6. Run evaluation (make sure queries.json exists)
evaluate_retrieval_bert(
    queries_json_path="data/eval/queries.json",
    vectors=vectors,
    filenames=filenames,
    k=5
)